In [1]:
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightEdges = BabyFlightNodes.read().split("%SECTION%")[2]
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightNodes = BabyFlightNodes.read().split("%SECTION%")[1]

In [2]:
with open('Flight1_HKG.txt', mode='r') as BabyFlight:
    Flight1 = BabyFlight.read().split("\n")[1]

In [3]:
from ipynb.fs.full.GetTrajectory import GetTrajectory
from ipynb.fs.full.PerpendicularDistance import NearestPointOnLine
from ipynb.fs.full.PerpendicularDistance import GetPerpendicularDistance
from ipynb.fs.full.PerpendicularDistance import EdgeNeighborSearch
from ipynb.fs.full.PerpendicularDistance import Haversine

In [4]:
FlightTrajectoryNodes = GetTrajectory(Flight1)

In [5]:
from ipynb.fs.full.GetTaxiway import GetTaxiway
from ipynb.fs.full.GetTaxiway import GetEdgeID
from ipynb.fs.full.GetTaxiway import GetEdgeStart
from ipynb.fs.full.GetTaxiway import GetEdgeEnd
from ipynb.fs.full.GetTaxiway import GetEdgeLength
from ipynb.fs.full.GetTaxiway import GetEdgeSpecification

In [6]:
FlightNodesList = GetTaxiway(FlightNodes, FlightEdges)[0]
FlightEdgesList = GetTaxiway(FlightNodes, FlightEdges)[1]
FlightEdgesWeights = GetTaxiway(FlightNodes, FlightEdges)[2]

In [7]:
FlightTrajectoryNodes

[('22.309341', '113.92628'),
 ('22.309263', '113.92652'),
 ('22.309158', '113.92673'),
 ('22.309032', '113.927'),
 ('22.30888', '113.92736'),
 ('22.30873', '113.927574'),
 ('22.308508', '113.92775'),
 ('22.308264', '113.927895'),
 ('22.308018', '113.927956'),
 ('22.307774', '113.92789'),
 ('22.307476', '113.92722'),
 ('22.30739', '113.92699'),
 ('22.307316', '113.926735'),
 ('22.307236', '113.9265'),
 ('22.307087', '113.92601'),
 ('22.306995', '113.92575'),
 ('22.30688', '113.9254'),
 ('22.306808', '113.92515'),
 ('22.306726', '113.924904'),
 ('22.306652', '113.924675'),
 ('22.306572', '113.92443'),
 ('22.306517', '113.92418'),
 ('22.306595', '113.923904'),
 ('22.307087', '113.923615'),
 ('22.307425', '113.92349'),
 ('22.307833', '113.923355'),
 ('22.308275', '113.92317'),
 ('22.308655', '113.92303'),
 ('22.30909', '113.92289'),
 ('22.30965', '113.92267'),
 ('22.310211', '113.92247'),
 ('22.31059', '113.922325'),
 ('22.312359', '113.921684'),
 ('22.312849', '113.921486'),
 ('22.313524'

In [17]:
#The Algorithm
for item in FlightTrajectoryNodes:
    print(item)
    #Iterate through the Edges for each Trajectory Node
    for index, edge in enumerate(FlightEdgesList):
        EdgeID = GetEdgeID(FlightEdgesList)[index]
        EdgeStart = GetEdgeStart(FlightEdgesList, FlightNodesList)[index]
        EdgeEnd = GetEdgeEnd(FlightEdgesList, FlightNodesList)[index]
        EdgeLength = GetEdgeLength(FlightEdgesList)[index]
        EdgeType = GetEdgeSpecification(FlightEdgesList)[index]
        
        PointX = float(item[0])
        PointY = float(item[1])
        StartX = float(EdgeStart[0])
        StartY = float(EdgeStart[1])
        EndX = float(EdgeEnd[0])
        EndY = float(EdgeEnd[1])
        
        #This will find edge neighbors to get input variables from
        if EdgeNeighborSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.0075) == True:
            #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            
            print(edge)
            print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance)
        else:
            pass

('22.309341', '113.92628')
('59', '1918', '377', '0', '11.962819189763287', 'taxiway', '')
12 59 22.3163615 113.9245665 22.3162869 113.9246504 (22.3162869, 113.9246504) 0.007134502293084366
('112', '1958', '137', '0', '55.80297276903443', 'taxiway', '')
32 112 22.3048632 113.9247428 22.3049925 113.9252673 (22.3049925, 113.9252673) 0.004464864336124801
('113', '137', '140', '0', '55.608842850566155', 'taxiway', '')
33 113 22.3049925 113.9252673 22.3050711 113.9258015 (22.3050711, 113.9258015) 0.004296627544947897
('114', '140', '143', '0', '56.358519384826174', 'taxiway', '')
34 114 22.3050711 113.9258015 22.3051074 113.9263483 (22.3051074, 113.9263483) 0.004234150900711066
('115', '143', '141', '0', '39.56816816292406', 'taxiway', '')
35 115 22.3051074 113.9263483 22.3050964 113.926733 (22.3051074, 113.9263483) 0.004234150900711066
('116', '141', '139', '0', '39.13324378135267', 'taxiway', '')
36 116 22.3050964 113.926733 22.3050617 113.9271118 (22.3050964, 113.926733) 0.00426870450605

KeyboardInterrupt: 